In [36]:
!pip install yfinance --upgrade --no-cache-dir
!pip install python-telegram-bot
import time
import telegram
import yfinance as yf
import pandas as pd
yf.pdr_override()

ticker = [
    'CXSE3.SA',
    'OIBR3.SA',
    'BRFS3.SA',
    'ITUB4.SA',
    'CIEL3.SA',
    'GOAU4.SA',
    'VALE3.SA',
    'EMBR3.SA',
    'PETR4.SA',
    'PGMN3.SA',
    'VIVR3.SA',
    'BRML3.SA'
]

for x in ticker:
    dia2 = yf.Ticker(x)
    Dia = dia2.history(period='1d', interval='5m').tail(1)
    Ano = yf.download(x, period='1y')
    print(x)     
    #ANO: criando coluna Data e transferindo valores
    Ano.insert(loc=0, column='DataMain', value=Ano.index)
    #remove a ultima data
    removedata = Ano.loc[(Ano['DataMain']==pd.to_datetime('today').normalize())]
    Ano = Ano.drop(removedata.index)
    #DIA: criando coluna Data e transferindo valores
    Dia.insert(loc=0, column='DataMain', value=Dia.index)
    #unindo os periodos
    Resumo = Ano.append(Dia)
    #ajuste da data
    Resumo['DataMain'] = pd.to_datetime(Resumo['DataMain'], utc=True).dt.date
    #resumindo as colunas desejadas (data e fechamento)
    ResumoEnd = Resumo[['DataMain', 'Close']]
    #calculo MACD
    rapidoMME = ResumoEnd.Close.ewm(span=12).mean()
    lentaMME = ResumoEnd.Close.ewm(span=26).mean()
    MACD = rapidoMME - lentaMME
    Sinal = MACD.ewm(span=9).mean()
    ResumoEnd['MACD'] = MACD
    ResumoEnd['Sinal'] = Sinal
    #ajuste index e retira data; coloca data como index
    ResumoEnd = ResumoEnd.set_index(pd.DatetimeIndex(ResumoEnd['DataMain'].values))
    ResumoEnd = ResumoEnd.drop('DataMain', 1)
    # criar código para verificar a compra ou venda
    ResumoEnd['flag'] = ''
    ResumoEnd['preco_compra'] = ''
    ResumoEnd['preco_venda'] = ''
    for i in range (1, len(ResumoEnd.Sinal)):
      if ResumoEnd['MACD'][i] > ResumoEnd['Sinal'][i]:
        if ResumoEnd['flag'][i-1] == 'C':
          ResumoEnd['flag'][i] = 'C'
        else:
          ResumoEnd['flag'][i] = 'C'
          ResumoEnd['preco_compra'][i] = ResumoEnd['Close'][i]
            
      elif ResumoEnd['MACD'][i] < ResumoEnd['Sinal'][i]:
        if ResumoEnd['flag'][i-1] == 'V':
            ResumoEnd['flag'][i] = 'V'
        else:
            ResumoEnd['flag'][i] = 'V'
            ResumoEnd['preco_venda'][i] = ResumoEnd['Close'][i]
            
    # mensagem ao TELEGRAM
    my_token = '1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I'
    chat_id = '-351556985'
    def envia_mensagem(msg, chat_id, token = my_token):
      bot = telegram.Bot(token = token)
      bot.sendMessage(chat_id = chat_id, text=msg)
    hoje = ResumoEnd.flag[-1]
    ontem = ResumoEnd.flag[-2]
    flag = hoje
    ticker2 = x[0:-3]
    site = f'https://www.google.com/search?q={ticker2}&rlz=1C1EJFC_enBR915BR916&oq={ticker2}&aqs=chrome..69i57j0l5j0i10i433j69i60.4408j0j7&sourceid=chrome&ie=UTF-8'
    preco_fechamento = round(ResumoEnd.Close.tail(1)[-1],2)
    msg = f'{ticker2},  {flag}  Preço de Fechamento: {preco_fechamento}'
    #verificar se houve alteração do sinal
    if ontem != hoje:
      envia_mensagem(msg, chat_id, my_token)
    time.sleep(2)

[*********************100%***********************]  1 of 1 completed
CXSE3.SA


<ipython-input-36-4e654e6c35a6>:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-4e654e6c35a6>:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-4e654e6c35a6>:71: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-4e654e6c35a6>:72: SettingWithCopyWarning:


A value is trying

AttributeError: module 'telegram' has no attribute 'Bot'